# YouTube Trending ETL: Exploring Global Cultural Preferences

This notebook processes daily trending YouTube video data from 113 countries to prepare it for analysis in Tableau. The objective of this project is to uncover global cultural preferences by examining patterns in video titles, sentiment, engagement metrics, and publishing behavior.

The full dataset is updated daily and sourced from Kaggle.

**Project Goals:**
- Download and validate the raw dataset
- Clean and standardize the data
- Engineer relevant features for analysis
- Export the processed dataset for visualization

Dataset source: https://www.kaggle.com/datasets/asaniczka/trending-youtube-videos-113-countries

## Environment Setup

In this section, we import all necessary Python libraries for data processing, text analysis, and emoji detection. We also configure the notebook to ensure that all file paths are relative to the project root directory, rather than the `notebooks/` folder. This allows consistent file organization when reading or writing to the `data/` and `processed/` directories.

All required dependencies are listed in the `requirements.txt` file in the root directory. To install them, run:

```bash
pip install -r requirements.txt

In [15]:
# Core Libraries
import pandas as pd
import numpy as np
import os
import csv
import warnings

# Suppress bottleneck version warning from pandas internals
warnings.filterwarnings("ignore", category=UserWarning, module="pandas.core.arrays.masked")

# Text Analysis
from textblob import TextBlob
import emoji

# Dataset Access
import kagglehub

# Ensure paths are relative to the project root, not the notebook location
from pathlib import Path

# Set working directory to project root (parent of /notebooks)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
os.chdir(project_root)

# Confirm working directory (optional)
# print("Working directory set to:", os.getcwd())

## Downloading the Dataset

We use the `kagglehub` library to programmatically download the latest version of the dataset from Kaggle. This ensures we always work with the most recent snapshot. The downloaded files will be cached locally, but for clarity and reproducibility, we will move the dataset into the `data/raw/` directory once downloaded.


In [2]:
# Download the dataset using kagglehub
dataset_path = kagglehub.dataset_download("asaniczka/trending-youtube-videos-113-countries")

# Examine dataset path (optional)
# print("Dataset downloaded to:", dataset_path)

# List the contents to verify
os.listdir(dataset_path)

['trending_yt_videos_113_countries.csv']

## Organizing the Raw Data

After downloading the dataset using `kagglehub`, we relocate the CSV file to a dedicated `data/raw/` directory. This improves reproducibility and organization by separating raw inputs from processed outputs. It also allows us to version and reference the dataset consistently across the project.

In [3]:
# Create raw data directory if it doesn't exist
raw_dir = os.path.join("data", "raw")
os.makedirs(raw_dir, exist_ok=True)

# Define source and destination paths
src_file = os.path.join(dataset_path, "trending_yt_videos_113_countries.csv")
dst_file = os.path.join(raw_dir, "trending_yt_videos_113_countries.csv")

# Copy the file to the data/raw/ directory
if not os.path.exists(dst_file):
    import shutil
    shutil.copy2(src_file, dst_file)
    print("Copied file to:", dst_file)
else:
    print("File already exists in:", dst_file)

File already exists in: data\raw\trending_yt_videos_113_countries.csv


## Loading and Previewing the Raw Data

We now load the raw CSV file into a Pandas DataFrame and inspect its structure. This step helps verify the format and identify any obvious issues such as missing values, inconsistent columns, or encoding problems.

In [4]:
# Load the dataset from the raw data directory
csv_file = os.path.join("data", "raw", "trending_yt_videos_113_countries.csv")
df = pd.read_csv(csv_file)

# Basic inspection
print("Dataset shape:", df.shape)
df.info()
df.head(3)

Dataset shape: (3594728, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594728 entries, 0 to 3594727
Data columns (total 18 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   title            object
 1   channel_name     object
 2   daily_rank       int64 
 3   daily_movement   int64 
 4   weekly_movement  int64 
 5   snapshot_date    object
 6   country          object
 7   view_count       int64 
 8   like_count       int64 
 9   comment_count    int64 
 10  description      object
 11  thumbnail_url    object
 12  video_id         object
 13  channel_id       object
 14  video_tags       object
 15  kind             object
 16  publish_date     object
 17  langauge         object
dtypes: int64(6), object(12)
memory usage: 493.7+ MB


,title,channel_name,daily_rank,daily_movement,weekly_movement,snapshot_date,country,view_count,like_count,comment_count,description,thumbnail_url,video_id,channel_id,video_tags,kind,publish_date,langauge
0,Master H - Haudi Kudiwa (Official Video) ft. T...,Master H,1,0,49,2025-07-25,ZW,1109150,32819,3103,Artist : @master_H_official Ft @Bottom_Camp ...,https://i.ytimg.com/vi/rgwL9V3t1og/mqdefault.jpg,rgwL9V3t1og,UC5jrU8WxzE16gPTnlqQAqew,#zimbabwe #music #trending #masterh #anika #da...,youtube#video,2025-07-04 00:00:00+00:00,NaN
1,EA SPORTS FC 26 | Official Reveal Trailer,EA SPORTS FC,2,0,23,2025-07-25,ZW,8365074,266900,22703,Innovation powered by you in every mode. The C...,https://i.ytimg.com/vi/TSi0iJYSQ24/mqdefault.jpg,TSi0iJYSQ24,UCoyaxd5LQSuP4ChkxK0pnZQ,"yt:cc=on, EA SPORTS FC, FC 25, EA FC, EA FC re...",youtube#video,2025-07-16 00:00:00+00:00,en
2,[LIVE] Manchester United vs Leeds United Pre-s...,Kampleng Com,3,0,47,2025-07-25,ZW,143492,617,3,[LIVE] Manchester United vs Leeds United Pre-s...,https://i.ytimg.com/vi/MjpunTTElhY/mqdefault.jpg,MjpunTTElhY,UCdMJwGhTUhAzk1ksH6iHjUw,"manchester united vs leeds united, manchester ...",youtube#video,2025-07-19 00:00:00+00:00,id


## Exploratory Data Analysis (EDA)

Before transforming the dataset, we perform a focused exploratory data analysis to better understand the data's structure, completeness, and potential issues. This step ensures that the data pipeline produces reliable outputs suitable for analysis and visualization.

### Key Checks Performed:

- **Missing Data**: We identify which columns contain missing values and their frequency. This helps determine whether any columns should be excluded or imputed in later stages.
  
- **Zero View Counts**: Since this dataset reflects trending YouTube videos, we investigate how many records—if any—report a view count of zero, which is unexpected for trending content. This may signal data freshness, delays in metric updates, or anomalies worth handling explicitly.

- **Duplicate Records**: We examine whether the same video appears multiple times by analyzing combinations of `video_id` with `snapshot_date` and `country`. This helps us distinguish between expected time-based snapshots and true duplicates that may need to be removed.

- **Uniqueness Metrics**: We compute counts of unique `video_id`s, `video_id + snapshot_date` pairs, and `video_id + country` pairs. These metrics provide insight into how videos propagate across countries and time, which is central to our analysis of global viewership patterns.

The results of this EDA guide our next step: deduplication and refinement of the dataset for reliable downstream use.

In [5]:
# Check for missing values
print("Missing values in full dataset:")
print(df.isnull().sum().sort_values(ascending=False).head(10))

# Identify videos with 0 views — an unexpected value for trending content
zero_view_rows = df[df["view_count"] == 0]
print(f"\nTotal rows with view_count == 0: {len(zero_view_rows)}")
display(zero_view_rows[["title", "channel_name", "snapshot_date", "country", "view_count", "like_count", "comment_count"]].head(10))

# Uniqueness and duplication checks
print("\nTotal rows:", len(df))

unique_videos = df["video_id"].nunique()
print("Unique video_ids:", unique_videos)

unique_video_day = df.drop_duplicates(subset=["video_id", "snapshot_date"]).shape[0]
print("Unique video_id + snapshot_date pairs:", unique_video_day)

unique_video_country = df.drop_duplicates(subset=["video_id", "country"]).shape[0]
print("Unique video_id + country pairs:", unique_video_country)

# Check for duplicates across video_id and snapshot_date
dupes = df.duplicated(subset=["video_id", "snapshot_date"]).sum()
print("Duplicates on video_id + snapshot_date:", dupes)


Missing values in full dataset:
video_tags       1227207
langauge          916563
description       692332
channel_name           0
publish_date           0
kind                   0
channel_id             0
video_id               0
thumbnail_url          0
title                  0
dtype: int64

Total rows with view_count == 0: 846


,title,channel_name,snapshot_date,country,view_count,like_count,comment_count
13140,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-23,MY,0,0,0
18784,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-22,MY,0,0,0
24427,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-21,MY,0,0,0
30075,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-20,MY,0,0,0
34320,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-19,TH,0,0,0
35717,🔴 [Live in HD] ONE Friday Fights 116: Adam vs....,ONE Championship,2025-07-19,MY,0,0,0
104178,ONE Friday Fights 114: Sangarthit vs. Suablack,ONE Championship,2025-07-07,LA,0,0,0
109803,ONE Friday Fights 114: Sangarthit vs. Suablack,ONE Championship,2025-07-06,LA,0,0,0
113286,🔴 [Live In HD] ONE Friday Fights 115: Rambong ...,ONE Championship,2025-07-05,TH,0,0,0
115434,ONE Friday Fights 114: Sangarthit vs. Suablack,ONE Championship,2025-07-05,LA,0,0,0



Total rows: 3594728
Unique video_ids: 274066
Unique video_id + snapshot_date pairs: 1338712
Unique video_id + country pairs: 714686
Duplicates on video_id + snapshot_date: 2256016


## Data Cleaning and Standardization

This section focuses on preparing the dataset for analysis by resolving common data quality issues. Specifically, we:
- Drop duplicate records based on `video_id`, `snapshot_date`, and `country`
- Remove rows with missing required fields
- Correct inconsistent column names
- Standardize text fields by trimming whitespace
- Convert date fields to proper datetime format

In [6]:
# Drop exact duplicate rows (if any)
df.drop_duplicates(inplace=True)

# # Drop duplicate trending entries for the same video in the same country on the same day
df.drop_duplicates(subset=["video_id", "snapshot_date", "country"], inplace=True)

# Drop rows missing critical fields (video title, country, or view count)
df.dropna(subset=["title", "country", "view_count"], inplace=True)

# Fix column name typo ("langauge" → "language")
df.rename(columns={"langauge": "language"}, inplace=True)

# Standardize string fields
df["title"] = df["title"].str.strip()
df["description"] = df["description"].fillna("").str.strip()
df["channel_name"] = df["channel_name"].str.strip()

### Parsing Date Columns

We convert the `snapshot_date` and `publish_date` fields to datetime format. This allows for accurate calculation of time-based features in the next stage of the pipeline.

In [7]:
# Parse date columns
df["snapshot_date"] = pd.to_datetime(df["snapshot_date"], errors="coerce")
df["publish_date"] = pd.to_datetime(df["publish_date"], utc=True, errors="coerce")

# Drop rows with unparseable dates, if any
df.dropna(subset=["snapshot_date", "publish_date"], inplace=True)

## Feature Engineering

In this section, we derive new features to capture patterns in title structure, engagement behavior, and publishing timelines. These features will support cross-country comparisons of content preferences and viewer interaction styles.

The engineered features include:
- Title characteristics (length, emoji usage, presence of a question)
- Sentiment of the title
- Engagement ratios (likes/views, comments/views)
- Temporal features (days between publish and trending, day of week)

In [8]:
# Title length (characters and word count)
df["title_length_chars"] = df["title"].str.len()
df["title_word_count"] = df["title"].str.split().str.len()

# Emoji presence in title
df["title_has_emoji"] = df["title"].apply(lambda x: any(char in emoji.EMOJI_DATA for char in x))

# Question mark in title
df["title_has_question"] = df["title"].str.contains(r"\?", regex=True)

# Sentiment polarity of title
df["title_sentiment"] = df["title"].apply(lambda x: TextBlob(x).sentiment.polarity)

In [9]:
# Avoid division errors
df["like_count"] = df["like_count"].replace(0, np.nan)
df["comment_count"] = df["comment_count"].replace(0, np.nan)
df["view_count"] = df["view_count"].replace(0, np.nan)

# Engagement ratios
df["like_ratio"] = df["like_count"] / df["view_count"]
df["comment_ratio"] = df["comment_count"] / df["view_count"]

# Drop rows where view_count is missing or zero after replacement
df.dropna(subset=["view_count"], inplace=True)

In [10]:
# Convert both to timezone-naive datetime
df["publish_date"] = df["publish_date"].dt.tz_localize(None)
df["snapshot_date"] = pd.to_datetime(df["snapshot_date"])  # already naive, but reparse just in case

# Compute days between publish and snapshot
df["days_since_publish"] = (df["snapshot_date"] - df["publish_date"]).dt.days

# Day of week published
df["publish_day_of_week"] = df["publish_date"].dt.day_name()

## Exporting the Processed Dataset

With all features engineered and the dataset cleaned, we now export the final data to a CSV file for use in Tableau. The exported file will include all video-level records, enriched with derived features such as title sentiment, engagement ratios, and temporal indicators.

To ensure compatibility with Tableau and avoid row-parsing issues, we take two key steps to handle special characters in text fields:

- **Normalizing line breaks**: Although quoting protects against row corruption, embedded newline characters in fields like `title` and `description` can still display inconsistently in Tableau. We replace those line breaks with spaces to maintain clean formatting in tooltips and tables.
- **Quoting all fields**: By using `quoting=csv.QUOTE_ALL`, we wrap every field in quotes. This ensures that commas, newlines, or special characters inside a field do not break the row/column structure expected by Tableau.

To support fast, high-level analysis in Tableau, we also generate an aggregated dataset using only the filtered countries included in the reduced dataset. This summary captures average values for key features at the country level, including title structure, engagement ratios, sentiment, and publishing behavior.

In [11]:
import csv

# Normalize line breaks
df["title"] = df["title"].replace({r"[\r\n]+": " "}, regex=True)
df["description"] = df["description"].replace({r"[\r\n]+": " "}, regex=True)

# Export with all fields quoted to preserve comma-containing columns
output_path_full = os.path.join("processed", "trending_youtube_enriched.csv")
df.to_csv(output_path_full, index=False, quoting=csv.QUOTE_ALL)

print("Exported full enriched dataset to:", output_path_full)

Exported full enriched dataset to: processed\trending_youtube_enriched.csv


In [12]:
# Aggregate selected metrics by country
agg_df = df.groupby("country").agg({
    "title_length_chars": "mean",
    "title_word_count": "mean",
    "title_has_emoji": "mean",
    "title_has_question": "mean",
    "title_sentiment": "mean",
    "like_ratio": "mean",
    "comment_ratio": "mean",
    "days_since_publish": "mean"
}).reset_index()

# Round numeric values for cleaner presentation in Tableau
agg_reduced_df = agg_df.round(4)

# Export the summary
summary_path = os.path.join("processed", "youtube_country_summary.csv")
agg_reduced_df.to_csv(summary_path, index=False)

print("Country-level summary exported to:", summary_path)

Country-level summary exported to: processed\youtube_country_summary.csv


## Verifying Exported Outputs

To ensure the ETL pipeline produced valid results, we reload the exported CSV files and confirm that they contain the expected structure and content. This includes checking the file shape, a sample of rows, and the presence of any missing values.

In [13]:
# Reload full dataset
processed_df = pd.read_csv("processed/trending_youtube_enriched.csv")
print("Full dataset shape:", processed_df.shape)
print("Missing values in full dataset:")
print(processed_df.isnull().sum().sort_values(ascending=False).head(10))
display(processed_df.head(2))

Full dataset shape: (3593882, 27)
Missing values in full dataset:
video_tags            1227048
language               916396
description            692332
like_ratio              47328
like_count              47328
comment_ratio           41935
comment_count           41935
title                       0
publish_date                0
days_since_publish          0
dtype: int64


,title,channel_name,daily_rank,daily_movement,weekly_movement,snapshot_date,country,view_count,like_count,comment_count,...,language,title_length_chars,title_word_count,title_has_emoji,title_has_question,title_sentiment,like_ratio,comment_ratio,days_since_publish,publish_day_of_week
0,Master H - Haudi Kudiwa (Official Video) ft. T...,Master H,1,0,49,2025-07-25,ZW,1109150.0,32819.0,3103.0,...,NaN,66,11,False,False,0.0,0.029589,0.002798,21,Friday
1,EA SPORTS FC 26 | Official Reveal Trailer,EA SPORTS FC,2,0,23,2025-07-25,ZW,8365074.0,266900.0,22703.0,...,en,41,8,False,False,0.0,0.031906,0.002714,9,Wednesday


## Appendix: Debugging CSV Export Issues for Tableau

To resolve unexpected row count inflation and malformed parsing in Tableau, we verified the raw CSV using Python file inspection tools. This helped confirm that the exported file maintained the correct structure (27 columns per row) and had properly quoted fields to avoid newlines or extra delimiters.

In [14]:
# Inspect raw lines from the CSV to verify line integrity
with open("processed/trending_youtube_enriched.csv", "r", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline())

# Check column counts in the raw CSV rows
import csv
with open("processed/trending_youtube_enriched.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(f"Line {i+1} has {len(row)} columns")
        if i == 4:
            break

"title","channel_name","daily_rank","daily_movement","weekly_movement","snapshot_date","country","view_count","like_count","comment_count","description","thumbnail_url","video_id","channel_id","video_tags","kind","publish_date","language","title_length_chars","title_word_count","title_has_emoji","title_has_question","title_sentiment","like_ratio","comment_ratio","days_since_publish","publish_day_of_week"

"Master H - Haudi Kudiwa (Official Video) ft. Tanaka Magazeni ANIKA","Master H","1","0","49","2025-07-25","ZW","1109150.0","32819.0","3103.0","Artist  : @master_H_official  Ft @Bottom_Camp  (TANAKA MAGAZENI ) Title : HAUDI KUDIWA  Staring  : @TheMusekaz  Producer : GHOST MAGICIAN  Composer : ​⁠@master_H_official ; @QOUNFUZEDOFFICIAL ; @Bottom_Camp  Directed by : @Mula_VIP_Rentals  Studio: BOTTOM CAMP 🎧 Year : 2025","https://i.ytimg.com/vi/rgwL9V3t1og/mqdefault.jpg","rgwL9V3t1og","UC5jrU8WxzE16gPTnlqQAqew","#zimbabwe #music #trending #masterh #anika #dancehall # Zimdancehall #hiphop #a